In [ ]:
from qiskit import QuantumCircuit
import numpy as np

# Create a 3-qubit circuit
qc = QuantumCircuit(3, 3)

qc.draw(output="mpl")


In [ ]:
theta = 1.5
varphi = 3.142

# Create the unknown state to be teleported using the u-gate.
qc.u(theta, varphi, 0.0, 0)

qc.barrier()

qc.draw(output="mpl")

In [ ]:

# Show the quantum state on a bloch sphere
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector

out_vector = Statevector(qc)

plot_bloch_multivector(out_vector)

In [ ]:
# Initialize the entangled pair to be used for teleportation
qc.h(1)
qc.cx(1, 2)
qc.barrier()

qc.draw(output="mpl")

In [ ]:
# Implement the teleportation protocol
# Use the CNOT gate and H gate to entangle the unknown state with state of qubit 1
qc.cx(0, 1)
qc.h(0)
qc.barrier()

# Measure the two qubits
qc.measure(0, 0)
qc.measure(1, 1)
qc.draw(output="mpl")



In [ ]:
# Apply corrections on Bob's end based on results of Alice's measurement
with qc.if_test((0, 1)):
    qc.z(2)
with qc.if_test((1, 1)):
    qc.x(2)
qc.barrier()

qc.draw(output="mpl")

In [ ]:
# Visual verification that the correct state has been teleported

from qiskit_aer import StatevectorSimulator

backend = StatevectorSimulator()
out_vector = backend.run(qc, shots=1).result().get_statevector()

plot_bloch_multivector(out_vector)



In [ ]:
# Verification through the U-inverse gate
# Apply the inverse of u-gate to verify that correct state has been teleported
qc.u(theta, varphi, 0.0, 2).inverse()
qc.measure(2, 2)

qc.draw(output="mpl")


In [ ]:
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Sampler
from qiskit.visualization import plot_histogram
from random import randint

# Take user input for number of shots
# nshots = int(input("Enter number of shots (1000-10000): "))
nshots = np.arange(1000, 11000, 1000)[randint(0, 9)]

# Backend selection
backend = AerSimulator()

# Transpilation
pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pass_manager.run(qc)

# Execution
sampler = Sampler(mode=backend)
job = sampler.run([isa_qc], shots=nshots)
result = job.result()

# Count extraction
counts = result[0].data.c.get_counts()
print(f"\nShots: {nshots}\n")
print(counts)

# Plotting count histogram
plot_histogram(counts)
